# Chapter 7: Building Chat Applications
## Github Models API Quickstart

This notebook is adapted from the [Azure OpenAI Samples Repository](https://github.com/Azure/azure-openai-samples?WT.mc_id=academic-105485-koreyst) that includes notebooks that access [Azure OpenAI](notebook-azure-openai.ipynb) services.


# അവലോകനം  
"വലിയ ഭാഷാ മോഡലുകൾ ടെക്സ്റ്റ് മുതൽ ടെക്സ്റ്റ് വരെ മാപ്പ് ചെയ്യുന്ന ഫംഗ്ഷനുകളാണ്. ഒരു ഇൻപുട്ട് ടെക്സ്റ്റ് സ്ട്രിംഗ് നൽകിയാൽ, വലിയ ഭാഷാ മോഡൽ അടുത്തതായി വരാനിരിക്കുന്ന ടെക്സ്റ്റ് പ്രവചിക്കാൻ ശ്രമിക്കുന്നു"(1). ഈ "ക്വിക്‌സ്റ്റാർട്ട്" നോട്ട്‌ബുക്ക് ഉപയോക്താക്കളെ ഉയർന്ന തലത്തിലുള്ള LLM ആശയങ്ങളിലേക്ക്, AML ആരംഭിക്കാൻ ആവശ്യമായ കോർ പാക്കേജ് ആവശ്യകതകൾക്ക്, പ്രോംപ്റ്റ് ഡിസൈനിലേക്ക് ഒരു സോഫ്റ്റ് പരിചയപ്പെടുത്തലിനും, വിവിധ ഉപയോഗ കേസുകളുടെ കുറച്ച് ചെറിയ ഉദാഹരണങ്ങൾക്കും പരിചയപ്പെടുത്തും.


## ഉള്ളടക്ക പട്ടിക  

[അവലോകനം](../../../../07-building-chat-applications/python)  
[OpenAI സേവനം എങ്ങനെ ഉപയോഗിക്കാം](../../../../07-building-chat-applications/python)  
[1. നിങ്ങളുടെ OpenAI സേവനം സൃഷ്ടിക്കൽ](../../../../07-building-chat-applications/python)  
[2. ഇൻസ്റ്റലേഷൻ](../../../../07-building-chat-applications/python)    
[3. ക്രെഡൻഷ്യലുകൾ](../../../../07-building-chat-applications/python)  

[ഉപയോഗ കേസുകൾ](../../../../07-building-chat-applications/python)    
[1. ടെക്സ്റ്റ് സംഗ്രഹിക്കുക](../../../../07-building-chat-applications/python)  
[2. ടെക്സ്റ്റ് വർഗ്ഗീകരിക്കുക](../../../../07-building-chat-applications/python)  
[3. പുതിയ ഉൽപ്പന്ന നാമങ്ങൾ സൃഷ്ടിക്കുക](../../../../07-building-chat-applications/python)  
[4. ഒരു ക്ലാസിഫയർ ഫൈൻ ട്യൂൺ ചെയ്യുക](../../../../07-building-chat-applications/python)  

[റഫറൻസുകൾ](../../../../07-building-chat-applications/python)


### നിങ്ങളുടെ ആദ്യ പ്രോംപ്റ്റ് നിർമ്മിക്കുക  
ഈ ചെറിയ അഭ്യാസം "സംഗ്രഹം" എന്ന ലളിതമായ ടാസ്കിനായി Github മോഡലുകളിൽ പ്രോംപ്റ്റുകൾ സമർപ്പിക്കുന്നതിന് ഒരു അടിസ്ഥാന പരിചയം നൽകും.


**പടികൾ**:  
1. നിങ്ങൾ ഇതുവരെ ചെയ്തിട്ടില്ലെങ്കിൽ, നിങ്ങളുടെ പൈത്തൺ പരിസ്ഥിതിയിൽ `azure-ai-inference` ലൈബ്രറി ഇൻസ്റ്റാൾ ചെയ്യുക.  
2. സ്റ്റാൻഡേർഡ് ഹെൽപ്പർ ലൈബ്രറികൾ ലോഡ് ചെയ്ത് Github മോഡലുകളുടെ ക്രെഡൻഷ്യലുകൾ സജ്ജമാക്കുക.  
3. നിങ്ങളുടെ ടാസ്കിനായി ഒരു മോഡൽ തിരഞ്ഞെടുക്കുക  
4. മോഡലിനായി ഒരു ലളിതമായ പ്രോംപ്റ്റ് സൃഷ്ടിക്കുക  
5. മോഡൽ API-യിലേക്ക് നിങ്ങളുടെ അഭ്യർത്ഥന സമർപ്പിക്കുക!


### 1. `azure-ai-inference` ഇൻസ്റ്റാൾ ചെയ്യുക


In [ ]:
%pip install azure-ai-inference

### 2. സഹായക ലൈബ്രറികൾ ഇറക്കുമതി ചെയ്ത് ക്രെഡൻഷ്യലുകൾ സൃഷ്ടിക്കുക


In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

### 3. ശരിയായ മോഡൽ കണ്ടെത്തൽ  
GPT-3.5-turbo അല്ലെങ്കിൽ GPT-4 മോഡലുകൾ സ്വാഭാവിക ഭാഷ മനസിലാക്കാനും സൃഷ്ടിക്കാനും കഴിയും.


In [ ]:
# Select the General Purpose curie model for text
model_name = "gpt-4o"

## 4. പ്രോംപ്റ്റ് ഡിസൈൻ  

"വലിയ ഭാഷാ മോഡലുകളുടെ മായാജാലം ഇതാണ്: വൻമാത്രത്തിലുള്ള വാചകങ്ങളിൽ ഈ പ്രവചന പിശക് കുറയ്ക്കാൻ പരിശീലനം ലഭിച്ചതിലൂടെ, മോഡലുകൾ ഈ പ്രവചനങ്ങൾക്ക് ഉപകാരപ്രദമായ ആശയങ്ങൾ പഠിക്കുന്നു. ഉദാഹരണത്തിന്, അവ പഠിക്കുന്ന ആശയങ്ങൾ ഇങ്ങനെ ആണ്"(1):

* എങ്ങനെ വാക്കുകൾ ശരിയായി എഴുതാം
* വ്യാകരണം എങ്ങനെ പ്രവർത്തിക്കുന്നു
* എങ്ങനെ പാരാഫ്രേസ് ചെയ്യാം
* എങ്ങനെ ചോദ്യങ്ങൾക്ക് ഉത്തരം നൽകാം
* എങ്ങനെ സംഭാഷണം നടത്താം
* എങ്ങനെ പല ഭാഷകളിലും എഴുതാം
* എങ്ങനെ കോഡ് ചെയ്യാം
* തുടങ്ങിയവ.

#### വലിയ ഭാഷാ മോഡൽ നിയന്ത്രിക്കുന്നത് എങ്ങനെ  
"വലിയ ഭാഷാ മോഡലിലേക്ക് നൽകുന്ന എല്ലാ ഇൻപുട്ടുകളിലെയും ഏറ്റവും സ്വാധീനശാലിയായത് ടെക്സ്റ്റ് പ്രോംപ്റ്റ് ആണ്(1).

വലിയ ഭാഷാ മോഡലുകൾക്ക് ചില മാർഗങ്ങളിൽ പ്രോംപ്റ്റ് നൽകി ഔട്ട്പുട്ട് ഉണ്ടാക്കാം:

Instruction: മോഡലിന് നിങ്ങൾ എന്ത് വേണമെന്ന് പറയുക  
Completion: നിങ്ങൾ തുടങ്ങിയത് മോഡൽ പൂർത്തിയാക്കാൻ പ്രേരിപ്പിക്കുക  
Demonstration: മോഡലിന് നിങ്ങൾ എന്ത് വേണമെന്ന് കാണിക്കുക, അതായത്:  
പ്രോംപ്റ്റിൽ കുറച്ച് ഉദാഹരണങ്ങൾ  
ഫൈൻ-ട്യൂണിംഗ് പരിശീലന ഡാറ്റാസെറ്റിൽ നൂറുകണക്കിന് ഉദാഹരണങ്ങൾ"



#### പ്രോംപ്റ്റുകൾ സൃഷ്ടിക്കാൻ മൂന്ന് അടിസ്ഥാന മാർഗ്ഗനിർദ്ദേശങ്ങൾ ഉണ്ട്:

**കാണിക്കുകയും പറയുകയും ചെയ്യുക**. നിങ്ങൾ എന്ത് വേണമെന്ന് നിർദ്ദേശങ്ങൾ, ഉദാഹരണങ്ങൾ, അല്ലെങ്കിൽ ഇരട്ടയടിക്കൂടിയുള്ള മാർഗ്ഗത്തിലൂടെ വ്യക്തമാക്കുക. മോഡലിന് ഒരു ലിസ്റ്റ് അക്ഷരമാലാനുസൃതമായി ക്രമീകരിക്കാനോ ഒരു പാരഗ്രാഫ് സന്റിമെന്റിന്റെ അടിസ്ഥാനത്തിൽ വർഗ്ഗീകരിക്കാനോ ആഗ്രഹിക്കുന്നുവെങ്കിൽ, അത് നിങ്ങൾക്ക് വേണ്ടതാണെന്ന് കാണിക്കുക.

**ഗുണമേൻമയുള്ള ഡാറ്റ നൽകുക**. നിങ്ങൾ ക്ലാസിഫയർ നിർമ്മിക്കാനോ മോഡലിനെ ഒരു പാറ്റേൺ പിന്തുടരാൻ പ്രേരിപ്പിക്കാനോ ശ്രമിക്കുന്നുവെങ്കിൽ, മതിയായ ഉദാഹരണങ്ങൾ ഉണ്ടെന്ന് ഉറപ്പാക്കുക. നിങ്ങളുടെ ഉദാഹരണങ്ങൾ പ്രൂഫ്‌റീഡ് ചെയ്യുക — മോഡൽ സാധാരണയായി അടിസ്ഥാന വാക്ക് തെറ്റുകൾ കാണുകയും മറുപടി നൽകുകയും ചെയ്യാൻ ബുദ്ധിമുട്ടില്ല, പക്ഷേ ഇത് ഉദ്ദേശപൂർവ്വമാണെന്ന് കരുതുകയും മറുപടിയിൽ സ്വാധീനം ചെലുത്തുകയും ചെയ്യാം.

**സജ്ജീകരണങ്ങൾ പരിശോധിക്കുക.** temperature, top_p സജ്ജീകരണങ്ങൾ മോഡൽ മറുപടി സൃഷ്ടിക്കുമ്പോൾ എത്ര നിർണ്ണായകമാണെന്ന് നിയന്ത്രിക്കുന്നു. ഒരേ ശരിയായ ഉത്തരം ഉള്ള ചോദ്യം ചോദിക്കുമ്പോൾ ഇവ കുറച്ച് സജ്ജീകരിക്കണം. കൂടുതൽ വൈവിധ്യമാർന്ന മറുപടികൾ ആഗ്രഹിക്കുന്നുവെങ്കിൽ ഇവ ഉയർത്താം. ഈ സജ്ജീകരണങ്ങളുമായി ബന്ധപ്പെട്ട് ആളുകൾ ചെയ്യുന്ന ഏറ്റവും വലിയ പിശക് ഇവ "ബുദ്ധിമുട്ട്" അല്ലെങ്കിൽ "സൃഷ്ടിപരമായ" നിയന്ത്രണങ്ങളായി കരുതുക എന്നതാണ്.


Source: https://learn.microsoft.com/azure/ai-services/openai/overview


### 5. സമർപ്പിക്കുക!


In [ ]:
# Create your first prompt
text_prompt = "Should oxford commas always be used?"

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

### അതേ കോൾ വീണ്ടും ചെയ്യുമ്പോൾ, ഫലങ്ങൾ എങ്ങനെ താരതമ്യം ചെയ്യുന്നു?


In [ ]:

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

## Summarize Text  
#### ചലഞ്ച്  
ഒരു ടെക്സ്റ്റ് പാസേജിന്റെ അവസാനം 'tl;dr:' ചേർത്ത് ടെക്സ്റ്റ് സംഗ്രഹിക്കുക. മോഡൽ എങ്ങനെ അധിക നിർദ്ദേശങ്ങൾ ഇല്ലാതെ നിരവധി ജോലികൾ നിർവഹിക്കാൻ കഴിയുന്നുവെന്ന് ശ്രദ്ധിക്കുക. മോഡലിന്റെ പെരുമാറ്റം മാറ്റാനും നിങ്ങൾക്ക് ലഭിക്കുന്ന സംഗ്രഹം ഇഷ്ടാനുസൃതമാക്കാനും tl;dr-നേക്കാൾ കൂടുതൽ വിവരണാത്മക പ്രോംപ്റ്റുകൾ ഉപയോഗിച്ച് പരീക്ഷണം നടത്താം(3).  

സമീപകാല പ്രവർത്തനങ്ങൾ വലിയ ടെക്സ്റ്റ് കോർപ്പസിൽ പ്രീ-ട്രെയിനിംഗ് ചെയ്ത് പിന്നീട് ഒരു പ്രത്യേക ജോലിയിൽ ഫൈൻ-ട്യൂണിംഗ് നടത്തുന്നതിലൂടെ നിരവധി NLP ജോലികളിലും ബെഞ്ച്മാർക്കുകളിലും വലിയ പുരോഗതി കാണിച്ചിട്ടുണ്ട്. സാധാരണയായി ആർക്കിടെക്ചറിൽ ജോലിക്ക് സ്വതന്ത്രമായിരുന്നാലും, ഈ രീതിക്ക് ആയിരക്കണക്കിന് അല്ലെങ്കിൽ പത്തിരണ്ടായിരം ഉദാഹരണങ്ങളുള്ള ജോലിക്ക് പ്രത്യേകമായ ഫൈൻ-ട്യൂണിംഗ് ഡാറ്റാസെറ്റുകൾ ആവശ്യമാണ്. അതിന്റെ വിപരീതമായി, മനുഷ്യർ സാധാരണയായി കുറച്ച് ഉദാഹരണങ്ങൾ അല്ലെങ്കിൽ ലളിതമായ നിർദ്ദേശങ്ങൾ മാത്രം ഉപയോഗിച്ച് ഒരു പുതിയ ഭാഷാ ജോലി നിർവഹിക്കാനാകും - ഇത് നിലവിലെ NLP സിസ്റ്റങ്ങൾ വലിയ തോതിൽ ബുദ്ധിമുട്ടുന്ന കാര്യമാണ്. ഇവിടെ ഞങ്ങൾ കാണിക്കുന്നത് ഭാഷാ മോഡലുകൾ വലുതാക്കുന്നത് ജോലിക്ക് സ്വതന്ത്രമായ കുറച്ച് ഉദാഹരണ പ്രകടനത്തെ വളരെ മെച്ചപ്പെടുത്തുന്നു, ചിലപ്പോൾ മുൻപ് മികച്ച ഫൈൻ-ട്യൂണിംഗ് സമീപനങ്ങളുമായി മത്സരിക്കാൻ പോലും സാധിക്കുന്നു.  

Tl;dr


# പല ഉപയോഗകേസുകൾക്കുള്ള വ്യായാമങ്ങൾ  
1. ടെക്സ്റ്റ് സംഗ്രഹിക്കുക  
2. ടെക്സ്റ്റ് വർഗ്ഗീകരിക്കുക  
3. പുതിയ ഉൽപ്പന്ന നാമങ്ങൾ സൃഷ്ടിക്കുക


In [ ]:
prompt = "Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something that current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches.\n\nTl;dr"


In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## ടെക്സ്റ്റ് വർഗ്ഗീകരിക്കുക  
#### ചലഞ്ച്  
ഇൻഫറൻസ് സമയത്ത് നൽകിയ വിഭാഗങ്ങളിലേക്ക് ഇനങ്ങളെ വർഗ്ഗീകരിക്കുക. താഴെ കൊടുത്ത ഉദാഹരണത്തിൽ, ഞങ്ങൾ പ്രോംപ്റ്റിൽ (*playground_reference) വർഗ്ഗങ്ങളും വർഗ്ഗീകരിക്കേണ്ട ടെക്സ്റ്റും നൽകുന്നു. 

ഉപഭോക്തൃ അന്വേഷണം: ഹലോ, എന്റെ ലാപ്ടോപ്പ് കീബോർഡിലെ ഒരു കീ അടുത്തിടെ തകരാറിലായി, അതിനായി ഒരു പകരം ആവശ്യമുണ്ട്:

വർഗ്ഗീകരിച്ച വിഭാഗം:


In [ ]:
prompt = "Classify the following inquiry into one of the following: categories: [Pricing, Hardware Support, Software Support]\n\ninquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement:\n\nClassified category:"
print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## പുതിയ ഉൽപ്പന്ന നാമങ്ങൾ സൃഷ്ടിക്കുക
#### വെല്ലുവിളി
ഉദാഹരണ വാക്കുകളിൽ നിന്ന് ഉൽപ്പന്ന നാമങ്ങൾ സൃഷ്ടിക്കുക. ഇവിടെ നാം നാമങ്ങൾ സൃഷ്ടിക്കാൻ പോകുന്ന ഉൽപ്പന്നത്തെക്കുറിച്ചുള്ള വിവരങ്ങൾ പ്രോംപ്റ്റിൽ ഉൾപ്പെടുത്തുന്നു. നാം ലഭിക്കേണ്ട മാതൃക കാണിക്കാൻ സമാനമായ ഒരു ഉദാഹരണം നൽകുന്നു. കൂടുതൽ സൃഷ്ടിപരമായ പ്രതികരണങ്ങൾക്കായി താപനില മൂല്യം ഉയർത്തിയിട്ടുണ്ട്.

ഉൽപ്പന്ന വിവരണം: ഒരു ഹോം മിൽക്‌ഷേക്ക് മേക്കർ  
സീഡ് വാക്കുകൾ: വേഗം, ആരോഗ്യകരം, സംക്ഷിപ്തം.  
ഉൽപ്പന്ന നാമങ്ങൾ: HomeShaker, Fit Shaker, QuickShake, Shake Maker

ഉൽപ്പന്ന വിവരണം: ഏതൊരു പാദ വലുപ്പത്തിലും പൊരുത്തപ്പെടുന്ന ഒരു ഷൂസിന്റെ ജോഡി.  
സീഡ് വാക്കുകൾ: അനുയോജ്യമായ, പൊരുത്തം, ഒംനി-ഫിറ്റ്.


In [ ]:
prompt = "Product description: A home milkshake maker\nSeed words: fast, healthy, compact.\nProduct names: HomeShaker, Fit Shaker, QuickShake, Shake Maker\n\nProduct description: A pair of shoes that can fit any foot size.\nSeed words: adaptable, fit, omni-fit."

print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt}])

response.choices[0].message.content

# റഫറൻസുകൾ  
- [Openai Cookbook](https://github.com/openai/openai-cookbook?WT.mc_id=academic-105485-koreyst)  
- [OpenAI Studio Examples](https://oai.azure.com/portal?WT.mc_id=academic-105485-koreyst)  
- [Best practices for fine-tuning GPT-3 to classify text](https://docs.google.com/document/d/1rqj7dkuvl7Byd5KQPUJRxc19BJt8wo0yHNwK84KfU3Q/edit#?WT.mc_id=academic-105485-koreyst)


# കൂടുതൽ സഹായത്തിനായി  
[OpenAI വാണിജ്യവത്കരണ സംഘം](AzureOpenAITeam@microsoft.com) 


# സംഭാവകർ
* [Chew-Yean Yam](https://www.linkedin.com/in/cyyam/)


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**അസൂയാ**:  
ഈ രേഖ AI വിവർത്തന സേവനം [Co-op Translator](https://github.com/Azure/co-op-translator) ഉപയോഗിച്ച് വിവർത്തനം ചെയ്തതാണ്. നാം കൃത്യതയ്ക്ക് ശ്രമിച്ചിട്ടുണ്ടെങ്കിലും, സ്വയം പ്രവർത്തിക്കുന്ന വിവർത്തനങ്ങളിൽ പിശകുകൾ അല്ലെങ്കിൽ തെറ്റുകൾ ഉണ്ടാകാമെന്ന് ദയവായി ശ്രദ്ധിക്കുക. അതിന്റെ മാതൃഭാഷയിലുള്ള യഥാർത്ഥ രേഖയാണ് പ്രാമാണികമായ ഉറവിടം എന്ന് പരിഗണിക്കേണ്ടതാണ്. നിർണായകമായ വിവരങ്ങൾക്ക്, പ്രൊഫഷണൽ മനുഷ്യ വിവർത്തനം ശുപാർശ ചെയ്യപ്പെടുന്നു. ഈ വിവർത്തനം ഉപയോഗിക്കുന്നതിൽ നിന്നുണ്ടാകുന്ന ഏതെങ്കിലും തെറ്റിദ്ധാരണകൾക്കോ തെറ്റായ വ്യാഖ്യാനങ്ങൾക്കോ ഞങ്ങൾ ഉത്തരവാദികളല്ല.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
